# 🔪 Resurface - Vulnerability Hunter Fine-Tuning

**Fully automated** - just click Runtime → Run All

Fine-tunes Llama 3.1 8B on 236 PortSwigger playbooks.

**Baseline:** Llama 3.3 70B without training = 25.9%

**Target:** 50%+ after fine-tuning

In [ ]:
#@title 1️⃣ Setup Environment (2-3 min)
!pip install -q transformers datasets peft accelerate bitsandbytes trl huggingface_hub
print("✅ Dependencies installed")

In [ ]:
#@title 2️⃣ Check GPU
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU: {gpu_name} ({gpu_mem:.1f} GB)")
else:
    print("❌ No GPU! Go to Runtime → Change runtime type → GPU")
    raise SystemExit("GPU required")

In [ ]:
#@title 3️⃣ Download Training Data from GitHub
import urllib.request
import os

# Download training data directly from GitHub
TRAIN_URL = "https://raw.githubusercontent.com/Ibrahim-sec/resurface/master/training/data/train.jsonl"

print("Downloading training data...")
urllib.request.urlretrieve(TRAIN_URL, "train.jsonl")

# Count examples
with open("train.jsonl") as f:
    num_examples = sum(1 for _ in f)
print(f"✅ Downloaded {num_examples} training examples")

In [ ]:
#@title 4️⃣ Load Training Data
import json
from datasets import Dataset

def load_training_data(filepath):
    examples = []
    with open(filepath) as f:
        for line in f:
            data = json.loads(line)
            # Convert to Llama chat format
            text = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{data['messages'][0]['content']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{data['messages'][1]['content']}<|eot_id|>"""
            examples.append({"text": text})
    return Dataset.from_list(examples)

dataset = load_training_data("train.jsonl")
print(f"✅ Loaded {len(dataset)} examples")
print(f"\n📝 Sample (first 300 chars):\n{dataset[0]['text'][:300]}...")

In [ ]:
#@title 5️⃣ Login to Hugging Face (for model access)
from huggingface_hub import login, HfFolder

# Check if already logged in
token = HfFolder.get_token()
if token:
    print("✅ Already logged in to Hugging Face")
else:
    print("🔑 Need Hugging Face token for Llama access")
    print("Get yours at: https://huggingface.co/settings/tokens")
    from getpass import getpass
    token = getpass("Enter HF token: ")
    login(token=token)
    print("✅ Logged in!")

In [ ]:
#@title 6️⃣ Load Model (3-5 min)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

# 4-bit quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print(f"📥 Loading {MODEL_ID}...")
print("(This takes 3-5 minutes, be patient)")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ Model loaded!")

In [ ]:
#@title 7️⃣ Configure LoRA
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

model.print_trainable_parameters()
print("✅ LoRA configured!")

In [ ]:
#@title 8️⃣ Setup Trainer
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./resurface-vuln-hunter",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    report_to="none",  # Disable wandb
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    args=training_args,
    dataset_text_field="text",
)

print("✅ Trainer ready!")

In [ ]:
#@title 9️⃣ 🚀 TRAIN! (30-60 min)
print("="*50)
print("🚀 STARTING FINE-TUNING")
print("="*50)
print("⏱️  This takes 30-60 minutes on T4")
print("📊 Watch the loss decrease below")
print("="*50)

trainer.train()

print("="*50)
print("✅ TRAINING COMPLETE!")
print("="*50)

In [ ]:
#@title 🔟 Save to Google Drive
from google.colab import drive
drive.mount('/content/drive')

SAVE_PATH = "/content/drive/MyDrive/resurface-vuln-hunter-lora"
trainer.model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

print(f"✅ Model saved to Google Drive: {SAVE_PATH}")
print("\n📥 You can download it from Drive and use in Resurface!")

In [ ]:
#@title 1️⃣1️⃣ Test the Model!
def generate_exploit(vuln_type, description):
    prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

You are a security researcher testing a web application for vulnerabilities.

**Vulnerability Type:** {vuln_type}
**Scenario:** {description}

Generate a step-by-step exploitation methodology. Include:
1. Detailed exploitation steps (numbered)
2. Key payloads to use
3. Indicators of success<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=800, temperature=0.3, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test on SQL injection
print("🧪 Testing on SQL Injection scenario...\n")
result = generate_exploit(
    "sqli",
    "The application has a product search feature that queries a database. The search term is reflected in the URL."
)
print(result)

# ✅ Done!

## Results
- Model saved to: `/content/drive/MyDrive/resurface-vuln-hunter-lora`
- Download from Google Drive

## Use in Resurface
```python
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
model = PeftModel.from_pretrained(base_model, "path/to/resurface-vuln-hunter-lora")
```

## Compare to Baseline
- **Before (no training):** 25.9%
- **After (fine-tuned):** Run eval to find out! 🎯